<a href="https://colab.research.google.com/github/mahdaviFarAsr/asr/blob/arcB/Language_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import another lib and connect to drive with download by google id

In [ ]:
from scipy.io import wavfile
from zipfile import ZipFile
import numpy as np
import os.path
import codecs
import json
import io
import os

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

def upload_file (folder_id, file_name):
    file_metadata = {'title': file_name, "parents": [{"id": folder_id, "kind": "drive#childList"}]}
    folder = drive.CreateFile(file_metadata)
    folder.SetContentFile(file_name) #The contents of the file
    folder.Upload()
    
def download_drive(partname,id_drive):
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  downloaded = drive.CreateFile({'id':id_drive}) # replace the id with id of file you want to access
  downloaded.GetContentFile(partname)

#Download my text

In [2]:
download_drive('mytext_2M.txt.gz','16qwCXUmxYIIrnnQwQvkbbGTRtEwdTZG_')

NameError: ignored

In [ ]:
!mv mytext_2M.txt.gz /tmp/

#Download kenlm source

In [2]:
!wget https://kheafield.com/code/kenlm.tar.gz
!tar xvzf kenlm.tar.gz

--2022-10-07 06:34:05--  https://kheafield.com/code/kenlm.tar.gz
Resolving kheafield.com (kheafield.com)... 35.196.63.85
Connecting to kheafield.com (kheafield.com)|35.196.63.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491888 (480K) [application/x-gzip]
Saving to: ‘kenlm.tar.gz’

kenlm.tar.gz        100%[===================>] 480.36K  2.89MB/s    in 0.2s    

2022-10-07 06:34:05 (2.89 MB/s) - ‘kenlm.tar.gz’ saved [491888/491888]

kenlm/
kenlm/include/
kenlm/include/util/
kenlm/include/util/string_piece.hh
kenlm/include/util/probing_hash_table.hh
kenlm/include/util/file_piece.hh
kenlm/include/util/file.hh
kenlm/include/util/read_compressed.hh
kenlm/include/util/ersatz_progress.hh
kenlm/include/util/pcqueue.hh
kenlm/include/util/usage.hh
kenlm/include/util/string_piece_hash.hh
kenlm/include/util/sorted_uniform.hh
kenlm/include/util/parallel_read.hh
kenlm/include/util/thread_pool.hh
kenlm/include/util/multi_intersection.hh
kenlm/include/util/exception.hh
k

#Install kenlm

In [ ]:
cd kenlm

/content/kenlm


In [ ]:
!mkdir -p build

In [ ]:
cd build

/content/kenlm/build


In [ ]:
!cmake ..

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Looking for pthread_create
-- Looking for pthread_create - not found
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threads: TRUE  
-- Boost version: 1.65.1
-- Found the following Boost libraries:


In [ ]:
!make -j 4

Scanning dependencies of target kenlm_filter
Scanning dependencies of target kenlm_util
[  1%] Building CXX object lm/filter/CMakeFiles/kenlm_filter.dir/phrase.cc.o
[  2%] Building CXX object lm/filter/CMakeFiles/kenlm_filter.dir/arpa_io.cc.o
[  3%] Building CXX object lm/filter/CMakeFiles/kenlm_filter.dir/vocab.cc.o
[  5%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/bignum-dtoa.cc.o
[  6%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/bignum.cc.o
[  7%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/cached-powers.cc.o
[  8%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/diy-fp.cc.o
[ 10%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/double-conversion.cc.o
[ 11%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/fast-dtoa.cc.o
[ 12%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/fixed-dtoa.cc.o
[ 13%] Building CXX object util/CMa

In [ ]:
cd bin

/content/kenlm/build/bin


#Create language model - step 1 - lm.binary

In [ ]:
import gzip
import io
import os
from urllib import request
data_upper = '/tmp/mytext_2M.txt.gz'
q=0
data_lower = '/tmp/lower.txt'
with open(data_lower, 'w', encoding='utf-8') as lower:
    with io.TextIOWrapper(io.BufferedReader(gzip.open(data_upper)), encoding='utf8') as upper:
        for line in upper:
          lower.write(line.lower())
os.remove(data_upper)

In [ ]:
lm_path = '/tmp/lm.arpa'
! ./lmplz --order 5 \
       --temp_prefix /tmp/ \
       --memory 50% \
       --text {data_lower} \
       --arpa {lm_path} \
       --prune 0 0 0 1

binary_path = '/tmp/lm.binary'
! ./build_binary -a 255 \
              -q 8 \
              trie \
              {lm_path} \
              {binary_path} 
os.remove(lm_path)

=== 1/5 Counting and sorting n-grams ===
Reading /tmp/lower.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
tcmalloc: large alloc 1200291840 bytes == 0x55feb8c02000 @  0x7f39ff2e81e7 0x55feb6812772 0x55feb67a6358 0x55feb6785290 0x55feb6771096 0x7f39fd481b97 0x55feb6772ada
tcmalloc: large alloc 5601337344 bytes == 0x55ff004b2000 @  0x7f39ff2e81e7 0x55feb6812772 0x55feb67fc7aa 0x55feb67fd1c8 0x55feb67852ad 0x55feb6771096 0x7f39fd481b97 0x55feb6772ada
****************************************************************************************************
Unigram tokens 36075978 types 353420
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:4241040 2:665613632 3:1248025600 4:1996840832 5:2912059648
tcmalloc: large alloc 2912067584 bytes == 0x55feb8c02000 @  0x7f39ff2e81e7 0x55feb6812772 0x55feb67fc7aa 0x55feb67fd1c8 0x55feb678584e 0x55feb6771096 0x7f39fd481b97 0x55feb6772ada
tcmalloc: large alloc 1996840960 bytes == 0x55

#Back to main dir

In [ ]:
cd /content/

/content


#Get Deepspeech model and install library


In [3]:
!wget https://github.com/mozilla/DeepSpeech/archive/v0.6.1.tar.gz
!tar xvzf v0.6.1.tar.gz

--2022-09-30 12:08:18--  https://github.com/mozilla/DeepSpeech/archive/v0.6.1.tar.gz
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/mozilla/DeepSpeech/tar.gz/refs/tags/v0.6.1 [following]
--2022-09-30 12:08:18--  https://codeload.github.com/mozilla/DeepSpeech/tar.gz/refs/tags/v0.6.1
Resolving codeload.github.com (codeload.github.com)... 20.205.243.165
Connecting to codeload.github.com (codeload.github.com)|20.205.243.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘v0.6.1.tar.gz’

v0.6.1.tar.gz           [    <=>             ]   4.76M  6.94MB/s    in 0.7s    

2022-09-30 12:08:19 (6.94 MB/s) - ‘v0.6.1.tar.gz’ saved [4991407]

DeepSpeech-0.6.1/
DeepSpeech-0.6.1/.cardboardlint.yml
DeepSpeech-0.6.1/.compute
DeepSpeech-0.6.1/.gitattributes
DeepSpeech-0.6.1

In [ ]:
!mv /tmp/lm.binary DeepSpeech-0.6.1

In [ ]:
cd DeepSpeech-0.6.1

/content/DeepSpeech-0.6.1


In [ ]:
#!pip uninstall tensorflow
#!pip install 'tensorflow-gpu==1.15.0'
!apt-get install sox
!apt-get install libsox-fmt-mp3
!sudo apt-get install sox
!pip3 install -r requirements.txt
!pip3 install $(python3 util/taskcluster.py --decoder)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3
Suggested packages:
  file libsox-fmt-all
The following NEW packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3 sox
0 upgraded, 8 newly installed, 0 to remove and 29 not upgraded.
Need to get 760 kB of archives.
After this operation, 6,715 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libopencore-amrnb0 amd64 0.1.3-2.1 [92.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libopencore-amrwb0 amd64 0.1.3-2.1 [45.8 kB]
Ign:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.3
Ign:4 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libm

In [ ]:
!python util/taskcluster.py --target native_client

#Create language model - step 2 - trie

In [ ]:
cd native_client

In [ ]:
!./generate_trie ../data/alphabet.txt /content/DeepSpeech-0.6.1/lm.binary trie

#Mount to drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Export to drive

In [ ]:
!mv trie /content/drive/My\ Drive/
!mv ../lm.binary /content/drive/My\ Drive/